## Electricity Spot Price Data in Norway (2022–2024)

This document presents an exploratory analysis and visualization of hourly electricity spot price data in Norway, covering the period from January 1, 2022, to December 31, 2024. The dataset includes actual recorded spot prices for each hour across Norway's five main electricity price regions:

- **NO1** (Oslo)
- **NO2** (Kristiansand)
- **NO3** (Trondheim)
- **NO4** (Tromsø)
- **NO5** (Bergen)

Each region has 26,000+ hourly records, corresponding to 24 hours × 365/366 days per year over three years (2022–2024), accounting for leap years.

The objectives of this document are as follows:

- To visualize the temporal structure and regional variation of spot prices
- To revisit and examine a previously developed Brownian Motion (BM) model applied to this dataset.
- To interactively experiment with the parameters of a Fractional Brownian Motion (fBM) model—particularly the Hurst parameter—and compare the behavior of simulated price paths with the actual market data.

By aligning simulated fBM trajectories with real spot price data, we aim to build insight into the modeling process and assess the suitability of memory-dependent stochastic models for representing electricity price dynamics in the Norwegian market.



### Electricity Price Visualization by Region

This interactive chart displays the time series of electricity prices (`eur_per_kwh`) for each Norwegian pricing zone (NO1 to NO5).

- The user can select a region using the dropdown menu.
- The corresponding CSV file is loaded dynamically from Google Drive.
- The data is sorted by `time_start` to ensure chronological plotting.



In [1]:
#from google.colab import drive
#from google.colab import auth

# Authenticate the user manually
#auth.authenticate_user()

# Mount Google Drive
#drive.mount('/content/drive')


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets

ModuleNotFoundError: No module named 'ipywidgets'

In [ ]:
regions = {
    "NO1": "NO1.csv",
    "NO2": "NO2.csv",
    "NO3": "NO3.csv",
    "NO4": "NO4.csv",
    "NO5": "NO5.csv"
}


def plot_region_data(region):
    path = f"spot_price_data/{regions[region]}"
    df = pd.read_csv(path, parse_dates=["time_start"])
    df = df.sort_values("time_start")

    # Plot
    plt.figure(figsize=(10, 4))
    plt.plot(df["time_start"], df["eur_per_kwh"], color='green', label="EUR/kWh")
    plt.title(f"EUR per kWh in {region}")
    plt.xlabel("Time")
    plt.ylabel("EUR per kWh")
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()

# Create interactive dropdown
region_dropdown = widgets.Dropdown(
    options=list(regions.keys()),
    description='Region:',
    value='NO1'
)

# Show the dropdown and connect it to the plotting function
widgets.interact(plot_region_data, region=region_dropdown);

interactive(children=(Dropdown(description='Region:', options=('NO1', 'NO2', 'NO3', 'NO4', 'NO5'), value='NO1'…

## 🔁 Simulation of a Mean-Reverting Process with Brownian Motion

In this section, we simulate a **mean-reverting stochastic process** driven by Gaussian white noise to model electricity spot prices. The model is based on the following stochastic differential equation (SDE):

$$
dX_t = \theta (m_t - X_t) \, dt + \sigma \, dW_t
$$

Where:
- $X_t$: The process (e.g., log-transformed electricity price)
- $m_t$: A time-dependent seasonal mean
- $\theta$: Mean-reversion speed (how strongly the process returns to the mean)
- $\sigma$: Volatility (standard deviation of random fluctuations)
- $W_t$: Standard Brownian motion (represented in discrete time by white noise)

---

### ⚙️ Discrete Approximation

In the simulation, this SDE is discretized using the **Euler-Maruyama method**:

$$
X_{t+1} = X_t + \theta (m_t - X_t) \, \Delta t + \varepsilon_t
$$

Where:
- $\Delta t$: Time step size
- $\varepsilon_t \sim \mathcal{N}(0, \Delta t)$: Gaussian white noise

---

### 📖 Reference
Vu, Le Quyen. (2023). Modeling Electricity Prices Using Stochastic Differential Equations. Master’s Thesis, University of Agder.
Available in the UiA Digital Library or via course materials in MA-138.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
import math

# Define your regions and file paths
regions = {
    "NO1": "NO1.csv",
    "NO2": "NO2.csv",
    "NO3": "NO3.csv",
    "NO4": "NO4.csv",
    "NO5": "NO5.csv"
}

# Seasonal mean function
def m(t):
    return  np.cos((2 * math.pi / (365*12)) * t) * .1 + .1

def est_theta (n, T, m, x):
  # Beregner snittreverteringshastigheten til prosessen
  t = np. linspace (0, T, n)
  teller = 0
  nevner = 0

  for i in range (len (x) - 1):
    teller = teller + (m[i] - x[i]) * (x[i+1] - x[i])
    nevner = nevner + (m[i] - x[i]) ** 2  * (t[i+1] - t[i])

  estimated_theta = teller / nevner
  return estimated_theta


# Estimate theta using seasonal mean
def estimate_theta(n, T, m_vals, x):
    t = np.linspace(0, T, n)
    numerator = 0
    denominator = 0

    for i in range(n - 1):
        dt = t[i+1] - t[i]
        numerator += (m_vals[i] - x[i]) * (x[i+1] - x[i])
        denominator += (m_vals[i] - x[i]) ** 2 * dt

    return numerator / denominator

# Estimate sigma using quadratic variation
def estimate_sigma(x, T):
    kvad = 0
    for i in range(len(x) - 1):
        kvad += (x[i+1] - x[i])**2
    estimated_sigma = np.sqrt(kvad / T)
    return estimated_sigma

# Simulate process using seasonal mean, estimated theta, and white noise
def simulate_process_with_data(region):
    path = f"spot_price_data/{regions[region]}"
    df = pd.read_csv(path, parse_dates=["time_start"])
    df = df.sort_values("time_start").reset_index(drop=True)

    actual = df["eur_per_kwh"].values

    n = len(actual)
    T = n


    t = np.linspace(0, T, n)
    m_vals = np.full(T, actual.mean())  # Mean-reverting level

    # Estimate theta and sigma from real data
    theta_hat = estimate_theta(n, T, m_vals, actual)
    sigma_hat = estimate_sigma(actual, T)

    # Simulate white noise
    WN = np.random.normal(0, np.sqrt(T / n), size=n)

    # Simulate process using estimated parameters and white noise
    x_sim = np.zeros(n)
    x_sim[0] = actual[0]
    for i in range(n - 1):
        dt = t[i+1] - t[i]
        x_sim[i + 1] = x_sim[i] + theta_hat * (m(i) - x_sim[i]) * dt + sigma_hat * WN[i]

    # Plot
    plt.figure(figsize=(12, 5))
    plt.plot(df["time_start"], actual, label=f"Actual Price ({region})", color='black')
    plt.plot(df["time_start"], x_sim, label=f"Simulated Process (θ={theta_hat:.4f}, σ={sigma_hat:.4f})", color='cyan', alpha=0.8)
    plt.title(f"Actual vs. Simulated Mean-Reverting Process for Region {region}")
    plt.xlabel("Time")
    plt.ylabel("EUR per kWh")
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()

    print(f"Estimated theta for {region}: {theta_hat:.6f}")
    print(f"Estimated sigma for {region}: {sigma_hat:.6f}")

# Interactive widget
region_dropdown = widgets.Dropdown(
    options=list(regions.keys()),
    description='Region:',
    value='NO1'
)

widgets.interact(simulate_process_with_data, region=region_dropdown);



interactive(children=(Dropdown(description='Region:', options=('NO1', 'NO2', 'NO3', 'NO4', 'NO5'), value='NO1'…

## Testing for Fractional Brownian Motion (fBM) Behavior in Electricity Price Data

In this section, we investigate whether the hourly electricity spot price data in each region of Norway exhibits statistical properties similar to those of **fractional Brownian motion (fBM)**.


> Reference: Jacod, J., Lejay, A., & Talay, D. (2018). [Statistical test for fractional Brownian motion based on detrending moving average algorithm](https://arxiv.org/abs/1803.08553). *arXiv:1803.08553*.

### Method

We use the **generalized second-order variation** as a diagnostic tool:

$$
V_n = \sum_{i=1}^{n-1} (X_{i+1} - 2X_i + X_{i-1})^2
$$

This variation measures the "roughness" of a time series. The behavior of this statistic changes with the **Hurst parameter** $ H $ of fBM:

- $ H = 0.5 $: standard Brownian motion (no memory, baseline variation).
- $ H < 0.5 $: anti-persistent behavior (more variation).
- $ H > 0.5 $: persistent behavior (smoother, less variation).

We compute this variation for the actual data and compare it with simulated fBM paths for a range of Hurst values from 0.1 to 0.9.

### Plot Explanation

- The **blue curve** shows the generalized variation of simulated fBM paths for various Hurst parameters.
- The **red dashed line** represents the variation of the actual electricity price data for the selected region.

### Suggested Hurst Value

The Hurst parameter whose simulated variation is **closest** to that of the real data is highlighted below the plot. This H value gives us an indication of the type of memory and roughness in the price data:

- If the best-fit H is significantly **greater than 0.5**, the price dynamics may exhibit **persistence** (long memory).
- If the best-fit H is **close to 0.5**, the data may behave similarly to standard Brownian motion.
- If the best-fit H is **less than 0.5**, the dynamics may be **rougher** or anti-persistent.

This information helps guide whether fBM is a suitable model for the spot price data and which Hurst parameter should be used in simulations or forecasting models.



In [ ]:
pip install fbm


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from fbm import FBM

# Define your regions and file paths
regions = {
    "NO1": "NO1.csv",
    "NO2": "NO2.csv",
    "NO3": "NO3.csv",
    "NO4": "NO4.csv",
    "NO5": "NO5.csv"
}

# Generalized second-order variation
def generalized_variation(x):
    x = np.asarray(x)
    increments = x[2:] - 2 * x[1:-1] + x[:-2]
    return np.sum(increments**2)

# Generate fBM path
def generate_fbm_series(n, hurst):
    f = FBM(n=n, hurst=hurst, length=1, method='daviesharte')
    return f.fbm()

# Plotting + fBM test with auto-H estimate
def test_region(region):
    path = f"spot_price_data/{regions[region]}"
    df = pd.read_csv(path, parse_dates=["time_start"])
    df = df.sort_values("time_start")
    series = df["eur_per_kwh"].values
    n = len(series)

    # Compute actual data variation
    data_var = generalized_variation(series)
    f= series.mean()

    # Simulate and compare variations for multiple Hurst values
    hurst_values = np.round(np.linspace(0.1, 0.9, 17), 2)
    fbm_vars = []

    for H in hurst_values:
        fbm_series = generate_fbm_series(n, hurst=H)*f
        var = generalized_variation(fbm_series)
        fbm_vars.append(var)

    # Find closest Hurst value
    diffs = np.abs(np.array(fbm_vars) - data_var)
    best_idx = np.argmin(diffs)
    best_H = hurst_values[best_idx]

    # Plot variation comparison
    plt.figure(figsize=(8, 4))
    plt.plot(hurst_values, fbm_vars, marker='o', label='fBM variation')
    plt.axhline(y=data_var, color='r', linestyle='--', label='Actual data variation')
    plt.title(f"Matching fBM Hurst for {region}")
    plt.xlabel("Hurst (H)")
    plt.ylabel("Generalized 2nd-Order Variation")
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()

    print(f"Generalized 2nd-order variation for {region}: {data_var:.4e}")
    print(f"✅ Closest match: H ≈ {best_H} (|Δ| = {diffs[best_idx]:.2e})")

# Interactive dropdown
region_dropdown = widgets.Dropdown(
    options=list(regions.keys()),
    description='Region:',
    value='NO1'
)

widgets.interact(test_region, region=region_dropdown);




interactive(children=(Dropdown(description='Region:', options=('NO1', 'NO2', 'NO3', 'NO4', 'NO5'), value='NO1'…

# 📘 Summary: Statistical Test for Fractional Brownian Motion (FBM) Using DMA

## 🧠 Author
**Grzegorz Sikora**  
Faculty of Pure and Applied Mathematics, Hugo Steinhaus Center  
Wrocław University of Science and Technology

---

## 🎯 Objective
This paper introduces a **rigorous statistical test** to determine whether a given time series is a trajectory of **fractional Brownian motion (FBM)**. The test is based on the **Detrending Moving Average (DMA)** statistic and its known distribution.

---

## 📐 Key Concepts

### Fractional Brownian Motion (FBM)
A generalization of Brownian motion characterized by the **Hurst exponent** $H$:
- $H = 0.5$: Standard Brownian motion
- $H < 0.5$: Anti-persistent (rougher)
- $H > 0.5$: Persistent (smoother)

---

## 📊 DMA Statistic

Given a time series $\{X(1), X(2), ..., X(N)\}$, the DMA statistic is defined as:

$$
\sigma^2(n) = \frac{1}{N - n} \sum_{j=n}^{N} \left(X(j) - \tilde{X}_n(j)\right)^2
$$

Where $\tilde{X}_n(j)$ is the simple moving average of $n$ past values:

$$
\tilde{X}_n(j) = \frac{1}{n} \sum_{k=0}^{n-1} X(j-k)
$$

---

## 📈 Statistical Test Procedure

### Null Hypothesis
$H_0$: The data follows FBM with parameters $H$ and $D$ (diffusion constant)

### Test Steps:
1. Compute $\sigma^2(n)$ for the data.
2. Construct the covariance matrix $\Sigma$ of the detrended process.
3. Calculate eigenvalues $\{\lambda_j\}$ of $\Sigma$.
4. Generate many ($L$) Monte Carlo samples:
   $$ \sigma^2_l(n) = \frac{1}{N-n} \sum_{j=1}^{N-n+1} \lambda_j U_j^{(l)} $$
   where $U_j^{(l)} \sim \chi^2(1)$.
5. Estimate the p-value:
   $$
   p = \frac{2 \min \left(\#\{ \sigma^2_l > t \}, \#\{ \sigma^2_l < t \} \right)}{L}
   $$
6. Reject $H_0$ if $p < \alpha$ (e.g., 0.05).

---

## 🧪 Monte Carlo Validation
- Simulated FBM with known Hurst exponents ($H = 0.25$ and $H = 0.75$).
- Showed **good test power** for subdiffusion ($H < 0.5$).
- Type II errors more common for superdiffusion ($H > 0.5$), suggesting test is **conservative**.

---

## 🛠 Implementation
The paper includes **Matlab code** for practical use of the test, including:
- DMA computation
- Covariance matrix construction
- Monte Carlo simulation of generalized chi-squared distribution

---

## 📖 Reference
Sikora, G. (2018). *Statistical test for fractional Brownian motion based on detrending moving average algorithm*.  
[arXiv:1803.08553](https://arxiv.org/abs/1803.08553)

---



In [ ]:
import numpy as np
from scipy.stats import chi2
import matplotlib.pyplot as plt

def dma_test(prices, n=10, H=0.5, D=1.0, alpha=0.05, L=1000, plot=True):
    """
    Python version of Grzegorz Sikora's DMA test for FBM.

    Parameters:
        prices: np.array, the empirical data
        n: int, DMA window size
        H: float, Hurst exponent
        D: float, scale parameter
        alpha: significance level
        L: number of Monte Carlo simulations
        plot: whether to plot the distribution

    Returns:
        h: int, 0 if H0 accepted, 1 if rejected
        p: float, p-value
        t: float, DMA statistic for empirical data
    """
    x = np.asarray(prices)
    N = len(x)

    # Step 1: compute DMA statistic t
    xmean = np.array([
        np.mean(x[j - n + 1: j + 1]) for j in range(n - 1, N)
    ])
    t = np.sum((x[n - 1:] - xmean)**2) / (N - n)

    # Step 2: construct FBM covariance matrix
    R = np.arange(1, N + 1)
    C = R[:, None]
    X = D * (R**(2*H) + C.T**(2*H) - np.abs(R[:, None] - R[None, :])**(2*H))

    # Covariance of Y process
    N = X.shape[0]
    size = N - n + 1
    Y1 = np.zeros((size, size))
    Y2 = np.zeros((size, size))
    Y3 = np.zeros((size, size))

    for i in range(size):
        for j in range(size):
            # Y1: term with (1 - 1/n)^2
            Y1[i, j] = (1 - 1/n)**2 * X[i + n - 1, j + n - 1]

            # Y2: mixed sum terms
            sum_row = np.sum(X[i + n - 1, j:j + n - 1])
            sum_col = np.sum(X[i:i + n - 1, j + n - 1])
            Y2[i, j] = (1 / n**2 - 1 / n) * (sum_row + sum_col)

            # Y3: full double sum
            Y3[i, j] = (1 / n**2) * np.sum(X[i:i + n - 1, j:j + n - 1])

    Y = Y1 + Y2 + Y3

    # Step 3: eigenvalues
    lambdas = np.linalg.eigvalsh(Y)
    lambdas = np.maximum(lambdas, 0)  # avoid negative due to numerical error

    # Step 4: simulate null distribution
    U_j = chi2.rvs(df=1, size=(len(lambdas), L))
    sigma_j = np.dot(lambdas, U_j) / (N - n)

    # Step 5: calculate p-value and test
    p = 2 * min(np.mean(sigma_j > t), np.mean(sigma_j < t))
    h = int(p < alpha)

    if plot:
        plt.hist(sigma_j, bins=50, alpha=0.6, label='Simulated Null')
        plt.axvline(t, color='red', linestyle='--', label='Empirical stat')
        plt.title(f'DMA Test for FBM (H={H}, D={D})')
        plt.xlabel('DMA Statistic')
        plt.ylabel('Frequency')
        plt.legend()
        plt.grid(True)
        plt.show()

    print(f"DMA Statistic: {t:.5f}")
    print(f"p-value: {p:.5f}")
    print("Reject H0 (not FBM)" if h else "Fail to reject H0 (may be FBM)")

    return h, p, t


In [ ]:
# Plotting + fBM test with auto-H estimate
def test_region_2(region):
    path = f"spot_price_data/{regions[region]}"
    df = pd.read_csv(path, parse_dates=["time_start"])
    df = df.sort_values("time_start")
    prices = df["nok_per_kwh"].values
    n = len(prices)



    #prices_std = (prices - np.mean(prices)) / np.std(prices)
    #prices_std = prices_std

    prices = prices[5500:6500]

    D_est = np.var(prices) / np.var(np.random.randn(len(prices)))
    # Compute actual data variation
    t_stat, p_value, result =  dma_test(prices, n=200, H=0.5, D=.02, alpha=0.05, L=1000, plot=True)


    #print(f"Generalized 2nd-order variation for {region}: {data_var:.4e}")
    #print(f"✅ Closest match: H ≈ {best_H}")

# Interactive dropdown
region_dropdown = widgets.Dropdown(
    options=list(regions.keys()),
    description='Region:',
    value='NO1'
)

widgets.interact(test_region_2, region=region_dropdown);

interactive(children=(Dropdown(description='Region:', options=('NO1', 'NO2', 'NO3', 'NO4', 'NO5'), value='NO1'…

## Interactive Simulation of a Mean-Reverting Fractional Brownian Motion (fBM) Model

In this section, we simulate a **mean-reverting stochastic process** driven by **fractional Brownian motion (fBM)** and compare it directly to the actual electricity spot price data for each Norwegian region.

This interactive model allows us to experiment with key parameters that control the dynamics of the simulated price path, providing valuable intuition for modeling and calibration.

---

### 🎯 Model Description

We simulate a process of the form:


$$
dX_t = \theta (u_t - X_t) \, dt + \sigma \, dB^H_t
$$

Where:
- $ X_t $: Simulated spot price at time $ t $
- $ u_t $: Long-term mean level (set to the average of the actual price series)
- $\theta $: **Mean reversion strength** – how strongly the process is pulled toward the mean
- $ \sigma $: **Volatility** – scales the amplitude of random movements
- $ B_H(t) $: **Fractional Brownian motion** with Hurst parameter $ H $
- $H $: Hurst parameter controlling memory:
  - $ H = 0.5 $: standard Brownian motion (no memory)
  - $ H > 0.5 $: persistent (smooth, long-term trends)
  - $ H < 0.5 $: anti-persistent (rough, noisy behavior)

---

### ⚙️ How to Use

Use the interactive sliders and dropdown menu to experiment with the following:

- **Region:** Choose between the five main Norwegian price regions (NO1 to NO5).
- **H (Hurst Parameter):** Controls the memory and smoothness of the noise.
- **σ (Sigma):** Controls the volatility of the simulated process.
- **θ (Theta):** Controls how strongly the process reverts to the long-term mean.

The simulation uses the actual price at time zero as the starting point and evolves forward using a combination of mean reversion and fractional noise.

---

### 📈 What the Plot Shows

- The **green curve** represents the actual spot price data for the selected region.
- The **orange curve** is the simulated path of the mean-reverting fBM process, given your selected parameters.

By adjusting the parameters, you can visually explore how well a mean-reverting fBM model can capture the behavior of real electricity prices. This is particularly useful for:

- **Model calibration**
- **Sensitivity analysis**
- **Understanding the role of memory (H), volatility (σ), and mean-reversion (θ)**

---

### 💡 Insights

- Try increasing $ H $ to simulate smoother trends and observe how the memory affects price evolution.
- Adjust $ \theta $ to see how quickly the simulated process returns to the long-term mean.
- Increase or decrease $ \sigma $ to match the variability in the actual price data.

This interactive tool helps build an intuitive understanding of how stochastic models can be tuned to approximate real-world electricity price dynamics.

**For the plot shown here,** we use a **constant target level** $u_t$ equal to the **mean of the actual electricity price data**. This ensures the simulated process fluctuates around a realistic, data-informed level.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from fbm import FBM
import ipywidgets as widgets

In [ ]:
def mean_function(t, n, f, mode):
    if mode == 'Zero':
        return np.zeros_like(t)
    elif mode == 'Linear':
        return t / n
    elif mode == 'Sine':
        return np.sin(12 * np.pi * t / n) * f
    elif mode == 'Step':
        return np.where(t < 15000, 0, 1) * f
    elif mode == 'mean':
        return np.ones(n) * f
    else:
        raise ValueError("Invalid mode")

def simulate_fbm_with_region(region, H=0.5, sigma=0.01, theta=0.1,  mu_mode='Sine'):
    # Load data
    path = f"spot_price_data/{regions[region]}"
    df = pd.read_csv(path, parse_dates=["time_start"])
    df = df.sort_values("time_start").reset_index(drop=True)

    actual = df["eur_per_kwh"].values
    time = df["time_start"]

    n = len(actual)
    dt = 1.0



    # Simulate mean-reverting fBM process
    X = np.zeros(n)
    X[0] = actual[0]


   # Target level u_t (can also use rolling mean here if you want)
    #u = np.full(n, actual.mean())  # or np.convolve(actual, np.ones(30)/30, mode='same')
    #u = np.sin(2 * np.pi * X) + X[0]
    f = actual.mean()
    t = np.arange(1, n + 1)
    mu_t = mean_function(t, n, f, mu_mode)


    # Generate fractional Brownian motion
    fbm_sim = FBM(n=n-1, hurst=H, length=n, method='daviesharte')
    B_H = fbm_sim.fbm()

    # Simulate mean-reverting fBM process
    X = np.zeros(n)
    for t in range(1, n):

        dBH = B_H[t] - B_H[t-1]
        X[t] = X[t-1] + theta * (mu_t[t-1] - X[t-1]) * dt + sigma * dBH


    # Plot
    plt.figure(figsize=(12, 5))
    plt.plot(time, actual, label=f"Actual Price ({region})", color='black')
    plt.plot(time, X, label=f"Simulated fBM (θ={theta}, σ={sigma}, H={H})", color='red', alpha=0.7)
    plt.plot(time, mu_t, '--', label='μ(t)', color='green')
    plt.title(f"Actual vs. Simulated fBM for Region {region}")
    plt.xlabel("Time")
    plt.ylabel("EUR per kWh")
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()

In [ ]:
widgets.interact(
    simulate_fbm_with_region,
    region=widgets.Dropdown(options=list(regions.keys()), description='Region:'),
    H=widgets.FloatSlider(min=0.1, max=0.95, step=0.05, value=0.5),
    sigma=widgets.FloatSlider(min=0, max=.3, step=0.001, value=0.01),
    theta=widgets.FloatSlider(min=0, max=.3, step=0.001, value=0.1),
    mu_mode = widgets.Dropdown(options=['Zero', 'Linear', 'Sine', 'Step', 'mean'], value='Sine', description='μ(t):')
);

interactive(children=(Dropdown(description='Region:', options=('NO1', 'NO2', 'NO3', 'NO4', 'NO5'), value='NO1'…